In [1]:
import json
import ipysheet as sheet
# !pip install ipywidgets
# !pip install ipysheet

In [14]:
def get_team_hp(team, units):
    return sum(max(0, u['hp']) for u in units if u['team'] == team)

def get_distance(u1, u2):
    return abs(u1['x'] - u2['x'])

def find_nearest_enemy(me, units):
    md, nearest = 9001, None
    for u in units:
        if u['hp'] <= 0 or u['team'] == me['team']:
            continue
        dx = get_distance(me, u)
        if dx < md:
            dx = md
            nearest = u
    return nearest

def battle(team1, team2, px1, px2, dt=0.1, logging=False):
    units = []
    for i, px, team in [[0, px1, team1], [1, px2, team2]]:
        for x, unit_params in zip(px, team):
            u = {k:v for k, v in unit_params.items()}
            u['hp'] = u['maxHealth']
            u['team'] = i
            u['ready'] = 0
            u['x'] = x
            units.append(u)
    time = 0
    for _ in range(900):
        if get_team_hp(0, units) <= 0 or get_team_hp(1, units) <= 0:
            break
        for un in units:
            if un['hp'] <= 0 or un['ready'] > time:
                continue
            enemy = find_nearest_enemy(un, units)
            if not enemy:
                continue
            if get_distance(un, enemy) <= un['attackRange']:
                if logging:
                    print(f'T{time:0.1f}: {un["type"]} team {un["team"]} hits {enemy["type"]} for {un["attackDamage"]}')
                enemy['hp'] -= un['attackDamage']
                un['ready'] = time + un['attackCooldown']
            else:
                x = un['x']
                un['x'] = x + [1, -1][un['team']] * (un['maxSpeed'] * dt)
                if logging:
                    print(f'T{time:0.1f}: {un["type"]} team {un["team"]} moves from {x} to {un["x"]}')

        time += dt
    return time, units

In [3]:
with open("data/GiantsGatesUnitsParams.json") as datafile:
    data = json.load(datafile)

RANGE_POWER_MULT = 1.7
    
for k in data.keys():
    data[k]["type"] = k

param_names = [
    'type',
    'cost',
    'maxSpeed',
    'maxHealth',
    'attackDamage',
    'attackCooldown',
    'attackRange']

column_names = param_names + ['dps', 'power']
unit_names = list(data.keys())
unit_sheet = sheet.sheet(
    columns=len(column_names),
    rows=len(unit_names),
#     row_headers=unit_names,
    column_headers=column_names,
    numeric_format='0')
data_rows = {}
for i, name in enumerate(unit_names):
    unit_data = data[name]
    cell_type = sheet.cell(i, 0, unit_data['type'])
    cell_cost = sheet.cell(i, 1, unit_data['cost'])
    cell_speed = sheet.cell(i, 2, unit_data['maxSpeed'])
    cell_health = sheet.cell(i, 3, unit_data['maxHealth'])
    cell_damage = sheet.cell(i, 4, unit_data['attackDamage'])
    cell_cooldown = sheet.cell(i, 5, unit_data['attackCooldown'])
    cell_range = sheet.cell(i, 6, unit_data['attackRange'])
    dps = unit_data['attackDamage'] / unit_data['attackCooldown']
    power = dps * unit_data['maxHealth'] / 5
    if unit_data['attackRange'] > 10:
        power *= RANGE_POWER_MULT
    cell_dps = sheet.cell(i, 7, dps, read_only=True, background_color='#dddddd')
    cell_power = sheet.cell(i, 8, power, read_only=True, background_color='#dddddd')
    data_rows[name] = [cell_type, cell_cost, cell_speed, cell_health, cell_damage, cell_cooldown, cell_range]
    
    @sheet.calculation(inputs=[cell_damage, cell_cooldown], output=cell_dps)
    def update_dps(a, b):
        return a / b
    
    @sheet.calculation(inputs=[cell_health, cell_damage, cell_cooldown, cell_range], output=cell_power)
    def update_power(h, d, c, r):
        p = h * d / c
        if r > 10:
            p *= RANGE_POWER_MULT
        return p
    
    

def get_unit_data(unit_type):
    
    res = {}
    for i, p in enumerate(param_names):
        res[p] = data_rows[unit_type][i].value
    return res

unit_sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='munchkin'), Cell(c…

In [22]:
munchkin = get_unit_data('munchkin')
thrower = get_unit_data('thrower')
ogre = get_unit_data('ogre')
shaman = get_unit_data('shaman')
brawler = get_unit_data('brawler')
warlock = get_unit_data('warlock')
headhunter = get_unit_data('headhunter')

t, result = battle([shaman, shaman, thrower], [headhunter], [0, 0, 0], [20], logging=False)

print(f'Time: {t}')
for un in result:
    print(f'{un["type"]} from team {un["team"]} health is {un["hp"]}')
    

Time: 3.3000000000000016
shaman from team 0 health is -10
shaman from team 0 health is -10
thrower from team 0 health is -5
headhunter from team 1 health is 135


In [23]:
out = {}
for name in unit_names:
    dd = get_unit_data(name)
    out[name] = dd
    out[name]['visualRange'] = 20
json.dumps(out)

'{"munchkin": {"type": "munchkin", "cost": 1, "maxSpeed": 12, "maxHealth": 100, "attackDamage": 10, "attackCooldown": 0.5, "attackRange": 3, "visualRange": 20}, "thrower": {"type": "thrower", "cost": 1, "maxSpeed": 15, "maxHealth": 50, "attackDamage": 15, "attackCooldown": 0.75, "attackRange": 20, "visualRange": 20}, "ogre": {"type": "ogre", "cost": 2, "maxSpeed": 10, "maxHealth": 200, "attackDamage": 19, "attackCooldown": 0.5, "attackRange": 4, "visualRange": 20}, "shaman": {"type": "shaman", "cost": 2, "maxSpeed": 12, "maxHealth": 100, "attackDamage": 25, "attackCooldown": 0.65, "attackRange": 20, "visualRange": 20}, "brawler": {"type": "brawler", "cost": 4, "maxSpeed": 8, "maxHealth": 400, "attackDamage": 40, "attackCooldown": 0.6, "attackRange": 5, "visualRange": 20}, "warlock": {"type": "warlock", "cost": 4, "maxSpeed": 9, "maxHealth": 200, "attackDamage": 50, "attackCooldown": 0.65, "attackRange": 20, "visualRange": 20}, "headhunter": {"type": "headhunter", "cost": 5, "maxSpeed":

In [18]:
munchkin = get_unit_data('munchkin')
thrower = get_unit_data('thrower')
ogre = get_unit_data('ogre')
shaman = get_unit_data('shaman')
brawler = get_unit_data('brawler')
warlock = get_unit_data('warlock')
headhunter = get_unit_data('headhunter')

battle_result = sheet.sheet(rows=7, columns=7, row_headers=unit_names, column_headers=unit_names)

for i, n1 in enumerate(unit_names):
    for j, n2 in enumerate(unit_names):
        unit1 = get_unit_data(n1)
        unit2 = get_unit_data(n2)
        t, result = battle([unit1], [unit2], [0], [20], logging=False)
        t = round(t, 2)
        hp1 = result[0]["hp"] / result[0]["maxHealth"]
        hp2 = result[1]["hp"] / result[1]["maxHealth"]
        color = ['#ff7f7f', '#7fff7f'][hp1 > hp2]
        if n1 == n2:
            color = '#dddddd'
        
        cell = sheet.cell(i, j, f'{max(hp1, hp2):0.1f} in {t}s', read_only=True, 
                          background_color=color)
#         print(f'Battle between {n1:>10} and {n2:<10} ended in {t} seconds. {hp1} : {hp2}')
        
        
battle_result


Sheet(cells=(Cell(column_end=0, column_start=0, read_only=True, row_end=0, row_start=0, style={'backgroundColo…

In [21]:
for i, n1 in enumerate(unit_names):
    for j, n2 in enumerate(unit_names):
        unit1 = get_unit_data(n1)
        unit2 = get_unit_data(n2)
        if unit1['cost'] >= unit2['cost']:
            continue
        q1 = unit2['cost'] // unit1['cost']
        team1 = [unit1 for _ in range(q1)]
        t, result = battle(team1, [unit2], [0 for _ in range(q1)], [20], logging=False)
        print('-' * 30)
        print(f'Battle between {q1} {n1} VS {n2} ends in {t:0.1f} seconds')
        for un in result:
            print(f'{un["type"]} from team {un["team"]} health is {un["hp"]} ({(un["hp"] / un["maxHealth"]):0.0%})')

------------------------------
Battle between 2 munchkin VS ogre ends in 6.8 seconds
munchkin from team 0 health is -14 (-14%)
munchkin from team 0 health is -14 (-14%)
ogre from team 1 health is 40 (20%)
------------------------------
Battle between 2 munchkin VS shaman ends in 5.0 seconds
munchkin from team 0 health is 0 (0%)
munchkin from team 0 health is 0 (0%)
shaman from team 1 health is 10 (10%)
------------------------------
Battle between 4 munchkin VS brawler ends in 8.0 seconds
munchkin from team 0 health is -20 (-20%)
munchkin from team 0 health is -20 (-20%)
munchkin from team 0 health is -20 (-20%)
munchkin from team 0 health is -20 (-20%)
brawler from team 1 health is 90 (22%)
------------------------------
Battle between 4 munchkin VS warlock ends in 5.0 seconds
munchkin from team 0 health is 0 (0%)
munchkin from team 0 health is 0 (0%)
munchkin from team 0 health is 0 (0%)
munchkin from team 0 health is 0 (0%)
warlock from team 1 health is 60 (30%)
--------------------